In [2]:
import pandas as pd
import numpy as np
import lxml.html as lh
import requests
from bs4 import BeautifulSoup

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [4]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
[len(C) for (title,C) in col]

[288, 288, 288]

In [7]:
#create a dataframe by first collecting the data in a dictionary
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


In [8]:
#drop boroughs with value 'Not assigned'
df =df[df['Borough']!='Not assigned']

In [9]:
#delete unwantwed characters "/n" from neighbourhoods
df.astype('str')
df['Neighbourhood\n'] = df['Neighbourhood\n'].str[:-1]
df.columns=['borough', 'neighbourhood','postalcode']
df.head()

,borough,neighbourhood,postalcode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A


In [13]:
# combine names of neighbourhoods in same postal code
Add_hoods = df.groupby('postalcode')['neighbourhood'].apply(lambda x: "%s" % ', '.join(x))

<class 'pandas.core.frame.DataFrame'>


borough          object
neighbourhood    object
postalcode       object
dtype: object

In [44]:
#create new dataframe for further operations
df2 = pd.DataFrame(Add_hoods)
df2.reset_index(inplace = True)
df2.shape

(103, 2)

In [50]:
#create a dataframe with borough names and unique postal codes
boroughs = df.drop('neighbourhood', axis = 1)
boroughs = boroughs.drop_duplicates()
boroughs.reset_index(inplace = True)
boroughs.shape

(103, 1)

In [59]:
#add borough column to the dataframe with postal_codes and neigbourhoods
df2['borough'] = boroughs['borough']

In [72]:
cols = ['postalcode', 'borough', 'neighbourhood']
toronto_postal_codes = df2[cols]
toronto_postal_codes.head()

,postalcode,borough,neighbourhood
0,M1B,North York,"Rouge, Malvern"
1,M1C,North York,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Downtown Toronto,"Guildwood, Morningside, West Hill"
3,M1G,North York,Woburn
4,M1H,Queen's Park,Cedarbrae


In [73]:
toronto_postal_codes.shape

(103, 3)

In [77]:
pip install geocoder

SyntaxError: invalid syntax (<ipython-input-77-bcb757b897b4>, line 1)